**Example:** 

One kmol of propane $\left(C_3H_{8}\right)$ is burned in air under rich conditions in an engine. Assuming rich, incomplete combustion (i.e., 30% of the propane goes to CO in the exhaust), determine (i)the equivalence ratio, and (ii)the combustion temperature.

$C_3H_8 + \phi \left(O_2 + 3.76 N_2\right) \rightarrow n_1CO_2 + n_2H_2O + n_3N_2 + n_4O_2 + n_5CO + n_6H_2$

where, $n_1 = 0.7*3$ and $n_5 = (1-0.7)*3$

(note in this example we're using propane becuase the default chemical mechanisms don't have C8 molecules in them ... also smaller mechanisms are faster).

In [1]:
# First we import fsolve from scipy
from scipy.optimize import fsolve
from math import exp
import numpy as np

# Set up the knowns
fuel = 1 #kmol of C3H6
C_fuel = 3
H_fuel = 8

# Note that we were told 70% of the octane converted to CO2 while 30% was converted to CO:
pB = 0.7 #70% burned to CO2

a = pB*(C_fuel+H_fuel/4)
a_s = (C_fuel+H_fuel/4)
phi = a_s/a
print("Phi is calculated to be %1.2f" % phi)

n1 = pB*(C_fuel)
n5 = (1-pB)*C_fuel

# Define our equations to solve
def eqn(vars):
    t, T, K, d1, a1, b1, c1, n2, n6, n3 = vars
    eq1 = t-T/1000
    eq2 = K - exp(2.743 - 1.761/t - 1.611/t**2 + 0.2803/t**3)
    eq3 = d1 - 2*a_s*(1-1/phi)
    eq4 = a1 - (1-K)
    eq5 = b1 - (H_fuel/2 + K*C_fuel - d1*(1-K))
    eq6 = c1 - (-C_fuel*d1*K)
    eq7 = K - n2*n5/(n1*n6)
    eq8 = n2 - (H_fuel/2 - d1 + n5)
    eq9 = n3 - 3.76*a_s/phi
    eq10 = n6 - (d1 - n5)
    return [eq1, eq2, eq3, eq4, eq5, eq6, eq7, eq8, eq9, eq10]

# Now call the fsolve function to solve our system of equatins
t, T, K, d1, a1, b1, c1, n2, n6, n3 = fsolve(eqn, (1, 1000, 1, 1, 1, 1, 1, 1, 1, 1))

print("The temperature is  %1.2f K" % T)
#print("n_CO2 = %1.2f" % n1)
#print("n_CO = %1.2f" % n5)

# Calculating mole fractions
ntot = n1+n2+n3+n5+n6 #total moles
x_CO2 = n1/ntot
x_CO = n5/ntot

print("x_CO2 = %1.4f" % x_CO2)
print("x_CO = %1.4f" % x_CO)

Phi is calculated to be 1.43
The temperature is  876.40 K
x_CO2 = 0.1042
x_CO = 0.0446


Alternatively, we could have specified the phi and the representative Temperature:

$\phi$ = 1.43

$T_{rxn}$ = 876 K

...in this case, we don't need a multi equation solver:


In [2]:
# Set up the knowns
fuel = 1 #kmol of C3H6
C_fuel = 3
H_fuel = 8

phi = 1.43
print("Phi is calculated to be %1.2f" % phi)

T = 876
t = T/1000
K = np.exp(2.743 - 1.761/t - 1.611/t**2 + 0.2803/t**3)
a1 = (1-K)
d1 = 2*a_s*(1-1/phi)
b1 = (H_fuel/2 + K*C_fuel - d1*(1-K))
c1 = (-C_fuel*d1*K)
n3 = 3.76*a_s/phi
n5 = ((-b1+(b1**2-4*a1*c1)**0.5)/(2*a1))
n1 = (C_fuel-n5)
n2 = (H_fuel/2 - d1 + n5)
n6 = (d1 - n5)

print("The temperature is  %1.2f K" % T)
#print("n_CO2 = %1.2f" % n1)
#print("n_CO = %1.2f" % n5)

# Calculating mole fractions
ntot = n1+n2+n3+n5+n6 #total moles
x_CO2 = n1/ntot
x_CO = n5/ntot

print("x_CO2 = %1.4f" % x_CO2)
print("x_CO = %1.4f" % x_CO)

Phi is calculated to be 1.43
The temperature is  876.00 K
x_CO2 = 0.1041
x_CO = 0.0448


%==================

Now let's check our answers with cantera:

In [3]:
# First you need to import the requried toolboxes
import cantera as ct

# Let's create the gas object
gas1 = ct.Solution('gri30.yaml')
gas1.set_equivalence_ratio(phi=phi, fuel='C3H8', oxidizer={'O2':1.0, 'N2':3.76}) #note we pulled "phi" from above
gas1.TP = T, 101325     #let's pull the temp from up above
#check that we've set the equivalence ratio correctly
g1eqr = gas1.equivalence_ratio()
print("The equivalence ratio prior to equilb calc is: %1.2f" % g1eqr)

# Solve the const enthaply equation
gas1.equilibrate('TP')

# Let's search for specific products
mf_co2 = gas1['CO2'].X
print("The mole fraction of CO2 is %1.4f " % mf_co2)
mf_co = gas1['CO'].X
print("The mole fraction of CO is %1.4f " % mf_co)

gas1()

The equivalence ratio prior to equilb calc is: 1.43
The mole fraction of CO2 is 0.1049 
The mole fraction of CO is 0.0434 

  gri30:

       temperature   876 K
          pressure   1.0132e+05 Pa
           density   0.36265 kg/m^3
  mean mol. weight   26.068 kg/kmol
   phase of matter   gas

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy       -1.9281e+06       -5.0262e+07  J
   internal energy       -2.2075e+06       -5.7545e+07  J
           entropy            8876.9         2.314e+05  J/K
    Gibbs function       -9.7043e+06       -2.5297e+08  J
 heat capacity c_p              1331             34696  J/K
 heat capacity c_v              1012             26381  J/K

                      mass frac. Y      mole frac. X     chem. pot. / RT
                     ---------------   ---------------   ---------------
                H2         0.0079364           0.10262           -19.463
                 H    

## i-Octane
Now, let's switch to a larger mechanism and try our iso-octane example:

In [4]:
# Let's try the same for iso-octane
T2 = 846
gas2 = ct.Solution('ic8_ver3_mech.yaml')
gas2.set_equivalence_ratio(phi=phi, fuel='IC8H18', oxidizer={'O2':1.0, 'N2':3.76}) #note we pulled "phi" from above
gas2.TP = T2, 101325     #let's pull the temp from up above
#check that we've set the equivalence ratio correctly
g2eqr = gas2.equivalence_ratio()
print("The equivalence ratio prior to equilb calc is: %1.2f" % g2eqr)

# Solve the const enthaply equation
gas2.equilibrate('TP')

# Let's search for specific products
mf_co2 = gas2['CO2'].X
print("The mole fraction of CO2 is %1.4f " % mf_co2)
mf_co = gas2['CO'].X
print("The mole fraction of CO is %1.4f " % mf_co)

gas2()

The equivalence ratio prior to equilb calc is: 1.43
The mole fraction of CO2 is 0.1142 
The mole fraction of CO is 0.0443 

  gas:

       temperature   846 K
          pressure   1.0132e+05 Pa
           density   0.38182 kg/m^3
  mean mol. weight   26.506 kg/kmol
   phase of matter   gas

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy       -1.9452e+06        -5.156e+07  J
   internal energy       -2.2106e+06       -5.8594e+07  J
           entropy            8708.3        2.3082e+05  J/K
    Gibbs function       -9.3124e+06       -2.4684e+08  J
 heat capacity c_p            1305.1             34593  J/K
 heat capacity c_v            991.43             26279  J/K

                      mass frac. Y      mole frac. X     chem. pot. / RT
                     ---------------   ---------------   ---------------
                 H        2.0768e-13        5.4611e-12           -9.7162
                H2      

C:\Users\baumgardner\AppData\Local\Temp\ipykernel_31180\1526225502.py:3: UserWarning: NasaPoly2::validate: 
For species EC6H13O2, discontinuity in s/R detected at Tmid = 1388.0
	Value computed using low-temperature polynomial:  107.80575085204809
	Value computed using high-temperature polynomial: 107.11626904383155

  gas2 = ct.Solution('ic8_ver3_mech.yaml')
